In [79]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm

In [80]:
df = pd.read_csv("dpo_requests.csv")
df.drop(columns="Unnamed: 0", axis=1, inplace=True)

In [81]:
checkpoint = "../Train/models/sft_clean_only_2"

In [82]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint).to("cuda")

In [83]:
def predict(cv: str, temp: float, is_temp: bool=False):
    input_ids = tokenizer(
        [cv],
        max_length=600,
        add_special_tokens=True,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )["input_ids"].to("cuda")

    if is_temp:
        output_ids = model.generate(
            input_ids=input_ids,
            no_repeat_ngram_size=4,
            temperature=temp,
            do_sample=True
        )[0]
    else:
        output_ids = model.generate(
            input_ids=input_ids,
            no_repeat_ngram_size=4
        )[0]

    summary = tokenizer.decode(output_ids, skip_special_tokens=True)

    return summary

In [84]:
# df["source"][2]

In [85]:
# predict(df["source"][2], 0.7)

In [86]:
# predict(df["source"][2], 1.4)

In [87]:
# predict(df["source"][2], 1.9)

In [88]:
about_07 = []
about_14 = []
about_19 = []
about = []
for id, row in tqdm(df[:1000].iterrows()):
    about.append(predict(row["source"], 1.0, is_temp=False))
    about_07.append(predict(row["source"], 0.7, True))
    about_14.append(predict(row["source"], 1.4, True))
    about_19.append(predict(row["source"], 1.9, True))
    

1000it [2:07:59,  7.68s/it]


In [89]:
df_gen = pd.DataFrame(data={"sft_clean_only_10epochs_no_temp": about,
                            "sft_clean_only_10epochs_temp_07": about_07,
                            "sft_clean_only_10epochs_temp_14": about_14,
                            "sft_clean_only_10epochs_temp_19": about_19
                            })

In [90]:
df_gen.to_csv("sft_clean_only_10epochs_from_0_to_999_with_temps.csv")